**Install hyperopt-sklearn**

In [ ]:
#!pip install git+https://github.com/hyperopt/hyperopt-sklearn

**Imports**

In [ ]:
import io
import numpy as np
import matplotlib.pyplot as plt

from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_regressor, linear_svr
from sklearn.model_selection import cross_val_score, train_test_split

**Covid dataset**

In [ ]:
X = np.loadtxt('../../datasets/covid/covid-vaccination-vs-death_ratio_input.data',delimiter=";")
y = np.loadtxt('../../datasets/covid/covid-vaccination-vs-death_ratio_classes.data',delimiter=";")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

estim_knn_c = HyperoptEstimator(regressor=k_neighbors_regressor("knn-reg"),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=10,
                          trial_timeout=300)

estim_lsvr_c = HyperoptEstimator(regressor=linear_svr("lsvr-reg"),
                          preprocessing=[],
                          algo=tpe.suggest,
                          max_evals=10,
                          trial_timeout=300)

estim_knn_c.fit(X_train, y_train)
estim_lsvr_c.fit(X_train, y_train)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(estim_knn_c.score(X_test, y_test))
print(estim_knn_c.best_model())

nmse_score = cross_val_score(estim_knn_c.best_model()["learner"], X, y, cv=5, scoring='neg_mean_squared_error')
r2_score = cross_val_score(estim_knn_c.best_model()["learner"], X, y, cv=5, scoring='r2')

knn_nmse = sum(nmse_score)/len(nmse_score)
knn_r2 = sum(r2_score)/len(r2_score)

print("KNN")
print("nmse:", knn_nmse)
print("r2:", knn_r2)

print(estim_lsvr_c.score(X_test, y_test))
print(estim_lsvr_c.best_model())

nmse_score = cross_val_score(estim_lsvr_c.best_model()["learner"], X, y, cv=5, scoring='neg_mean_squared_error')
r2_score = cross_val_score(estim_lsvr_c.best_model()["learner"], X, y, cv=5, scoring='r2')

linSVR_nmse = sum(nmse_score)/len(nmse_score)
linSVR_r2 = sum(r2_score)/len(r2_score)

print("LinearSVR")
print("nmse:", linSVR_nmse)
print("r2:", linSVR_r2)

allScores = []
allScores.append(linSVR_nmse)
allScores.append(knn_nmse)
allScores.append(linSVR_r2)
allScores.append(knn_r2)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])

x = ['LinearSVR_nmse', 'KNN_nmse','LinearSVR_r2', 'KNN_r2']
y = allScores

ax.bar(x,y)
fig.suptitle('Hyperopt Covid', fontsize=16)
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.show()